In [27]:
import os
import xarray as xr
import argparse
from rdflib import Graph, Literal, Namespace, RDF, URIRef, XSD, SOSA
from rdflib.graph import DATASET_DEFAULT_GRAPH_ID as default
import json
from shapely.geometry import Point

In [28]:
def coordinates_to_wkt(x, y):
    "convert point coordinates to wkt literal"
    point = Point(x, y)
    wkt = point.wkt
    return wkt

def create_sensor_rdf(path_to_geojson: str, path_to_rdf_output: str):
    """convert sensor community data (sensors and locations) geojson to rdf file"""

    # Create a graph
    g = Graph()
    g.bind("aqqa", aqqa)


    path_to_geojson = path_to_geojson
    with open(path_to_geojson, 'r') as geojson_file:
        data = json.load(geojson_file)

    for sensor in data["features"]:
        
        sensor_id = sensor["properties"]["sensor_id"]
        sensor_coords = sensor["geometry"]["coordinates"]
        sensor_geom_wkt = coordinates_to_wkt(sensor_coords[0], sensor_coords[1])
    
        ent_sensor = URIRef(aqqa[f"sensor?id={sensor_id}"])
        ent_sensor_geom = URIRef(aqqa[f"sensor_location_geom?id={sensor_id}"])
        ent_has_provider = URIRef(aqqa["hasProvider"]) 
    
        g.add((ent_sensor, RDF.type, SOSA.Sensor))
        g.add((ent_sensor, geo.hasGeometry, ent_sensor_geom))
        g.add((ent_sensor, aqqa.hasProvider, Literal("Sensor Community API")))
        g.add((ent_sensor_geom, RDF.type, sf.Geometry))
        g.add((ent_sensor_geom, geo.asWKT, Literal(sensor_geom_wkt, datatype=geo.wktLiteral)))
    
        g.serialize(destination=path_to_rdf_output)

In [29]:
# Constants
OBSERVABLE_PROPERTIES_JSON_PATH = "/workspaces/aqqa-kg-creation-dev/ontology/observableProperties.json"
NAMESPACES_JSON_PATH = "/workspaces/aqqa-kg-creation-dev/ontology/namespaces.json"

def load_json_file(file_path):
    with open(file_path) as json_file:
        return json.load(json_file)

# Load dictionaries
variables_dict = load_json_file(OBSERVABLE_PROPERTIES_JSON_PATH)
namespaces_dict = load_json_file(NAMESPACES_JSON_PATH)

# Define the namespaces
namespace_mapping = {namespace["prefix"]: namespace["uri"] for namespace in namespaces_dict.get("namespaces", [])}
aqqa = Namespace(namespace_mapping.get("aqqa", None))
geo = Namespace(namespace_mapping.get("geo", None))
xsd = Namespace(namespace_mapping.get("xsd", None))
sf = Namespace(namespace_mapping.get("sf", None))

In [30]:
path_to_geojson = "/mnt/data/processed/Sensor_Community/sample.geojson"
path_to_output = "/mnt/data/processed/RDF/Sensor_Community/sensor_locations.ttl"
create_sensor_rdf(path_to_geojson, path_to_output)

In [33]:
# Create a graph
g = Graph()
g.bind("aqqa", aqqa)

provider = "SensorCommunityAPI"

path_to_geojson = path_to_geojson
with open(path_to_geojson, 'r') as geojson_file:
    data = json.load(geojson_file)

for sensor in data["features"]:
    
    sensor_id = sensor["properties"]["sensor_id"]
    sensor_coords = sensor["geometry"]["coordinates"]
    sensor_geom_wkt = coordinates_to_wkt(sensor_coords[0], sensor_coords[1])

    ent_sensor = URIRef(aqqa[f"sensor?id={sensor_id}"])
    ent_foi = URIRef(aqqa[f"sensor_location?id="])
    ent_sensor_geom = URIRef(aqqa[f"sensor_location_geom?id={sensor_id}"])
    ent_has_provider = URIRef(aqqa["hasProvider"]) 

    #g.add((ent_sensor, RDF.type, SOSA.Sensor))
    #g.add((ent_sensor, geo.hasGeometry, ent_sensor_geom))
    #g.add((ent_sensor, aqqa.hasProvider, Literal(provider)))
    #g.add((ent_sensor_geom, RDF.type, sf.Geometry))
    #g.add((ent_sensor_geom, geo.asWKT, Literal(sensor_geom_wkt, datatype=geo.wktLiteral)))

    for observation in sensor["properties"]["values"]:
        timestamp, pm10, pm25 = observation
        print(sensor_id, timestamp, pm25)

        ent_observation_pm25 = URIRef(aqqa[f"provider={provider}&sensor_id={sensor_id}&var=pm25&time={timestamp}"])
        ent_observation_pm10 = URIRef(aqqa[f"provider={provider}&sensor_id={sensor_id}&var=pm10&time={timestamp}"])
        
        

    #g.serialize(destination=path_to_rdf_output)

17259 2022-01-01 11.135950266429841
17259 2022-01-14 26.947619893428065
17259 2022-01-07 9.649614711033275
17259 2022-01-24 8.325009041591318
17259 2022-01-09 4.980316344463972
17259 2022-01-18 14.925315315315315
17259 2022-01-20 5.869098786828423
17259 2022-01-17 20.2287610619469
17259 2022-01-23 21.362359346642467
17259 2022-01-08 2.505925925925926
17259 2022-01-03 2.238351449275362
17259 2022-01-04 2.500742049469965
17259 2022-01-15 20.30319298245614
17259 2022-01-11 15.711699115044249
17259 2022-01-21 5.917490909090909
17259 2022-01-02 10.326714801444043
17259 2022-01-19 10.934573502722323
17259 2022-01-05 2.202783687943262
17259 2022-01-13 22.731319073083778
17259 2022-01-10 21.08254766031196
17259 2022-01-16 24.905107913669063
17259 2022-01-12 13.72458110516934
17259 2022-01-22 14.247982142857142
17259 2022-01-06 8.734922010398613
17259 2022-01-27 12.449648241206031
17259 2022-01-25 23.76801801801802
17259 2022-01-26 15.779458483754512
17259 2022-01-29 6.660924956369983
17259 202

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

